In [4]:
# tSZ Map Painter (Notebook Version)
# ----------------------------------
# Serial version of the original MPI-enabled script, adapted for
# interactive use in a Jupyter notebook.  To run:
#   1. Ensure all required packages are installed in your Julia environment.
#   2. Define/adjust `idx_range`, `cat_dir`, and `out_dir` as needed.
#   3. Execute the cells sequentially.
using XGPaint
using CSV, DataFrames
using Healpix
using Pixell
using Dates, Printf

In [5]:
# ---------------------------------------------------------------------------
# Main worker: load a halo catalogue, paint the corresponding y‑map, and save.
# Automatically skips if either the catalogue or parameter file is missing.
# NO BEAM CONVOLUTION in this version.
# ---------------------------------------------------------------------------
wall() = time()
function process_catalogue_fast(cat_path::AbstractString,
    par_path::AbstractString,
    idx::Integer)

    # ---------------- safety checks -----------------------------------------
    if !isfile(cat_path)
    @warn "Catalogue file $(cat_path) not found – skipping idx=$(idx)."
    return nothing
    end
    if !isfile(par_path)
    @warn "Parameter file $(par_path) not found – skipping idx=$(idx)."
    return nothing
    end

    println("\nProcessing catalogue: $(basename(cat_path)) (idx = $idx)")

    # ---------------- 1 | read catalogue ------------------------------------
    df        = CSV.read(cat_path, DataFrame)
    z_arr     = df.z
    m_arr     = df.M .* 1e14                        # Msun
    ra_arr    = rem.(df.lon .+ π, 2π) .- π          # convert to [−π, π]
    dec_arr   = df.lat .- π/2                       # convert to [−π/2, π/2]

    # ---------------- 2 | cosmology -----------------------------------------
    p         = CSV.read(par_path, DataFrame)[1, :]
    h, Ωb, Ωc, B = p.h,  p.Ob0h2/p.h^2,  p.Oc0h2/p.h^2,  p.B
    @printf("   cosmo: h = %.3f   Ω_b = %.4f   Ω_c = %.4f\n", h, Ωb, Ωc)

    # ---------------- 3 | build y‑profile interpolator (NO BEAM) -------------
    t0 = wall()
    a10_base = Arnauld10ThermalSZProfile(Omega_c = Ωc, Omega_b = Ωb, h = h, B = B)
    # Supported kwargs in your a10_fast_profile.jl are (Nx, x_min, x_max).
    y_model  = build_interpolator(a10_base; Nx = 256)
    @printf("   interpolator built in %.3f s\n", wall() - t0)

    # ---------------- 4 | prepare canvas patch ------------------------------
    box        = [ 5  -5; -5  5] * Pixell.degree    # 10°×10° patch (RA, Dec)
    shape, wcs = geometry(CarClenshawCurtis{Float64}, box, 0.5 * Pixell.arcminute)
    sky_map    = Enmap(zeros(shape), wcs)
    workspace  = profileworkspace(shape, wcs)

    # ---------------- 5 | paint all clusters --------------------------------
    t1 = wall()
    paint!(sky_map, workspace, y_model, m_arr, z_arr, ra_arr, dec_arr)
    @printf("   painting finished in %.3f s\n", wall() - t1)

    # ---------------- 6 | write result to FITS ------------------------------
    out_dir  = "../map_patches_demo"              # change if desired
    mkpath(out_dir)
    mapfile = joinpath(out_dir, @sprintf("y_map_10deg_fast_%d.fits", idx))

    write_map(mapfile, sky_map)
    println("   saved → $mapfile")
end

process_catalogue_fast (generic function with 1 method)

In [6]:
# ---------------------------------------------------------------------------
# Example driver loop.  Adjust `idx_range` (and optionally the directories)
# to select which catalogues you wish to process.
# ---------------------------------------------------------------------------
idx_range = 0:1
cat_dir   = "../catalogue_demo"      # <- make sure this is the right folder

for idx in idx_range
    par_file = joinpath(cat_dir, @sprintf("params_sbi_%d.csv",    idx))
    cat_file = joinpath(cat_dir, @sprintf("catalogue_sbi_%d.csv", idx))

    process_catalogue_fast(cat_file, par_file, idx)
end

println("\nAll requested catalogues processed.")




Processing catalogue: catalogue_sbi_0.csv (idx = 0)
   cosmo: h = 0.650   Ω_b = 0.0516   Ω_c = 0.2764
   interpolator built in 1.295 s
   painting finished in 5.214 s
   saved → ../map_patches_demo/y_map_10deg_fast_0.fits

Processing catalogue: catalogue_sbi_1.csv (idx = 1)
   cosmo: h = 0.892   Ω_b = 0.0308   Ω_c = 0.1581
   interpolator built in 0.026 s
   painting finished in 11.270 s
   saved → ../map_patches_demo/y_map_10deg_fast_1.fits

All requested catalogues processed.
